<a href="https://colab.research.google.com/github/ShesterG/Sentiment_Tools_Evaluation/blob/master/scratch_work/sentiment_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Constants

In [1]:
!git clone https://github.com/ShesterG/Stance-Detection-Ghana-2020-Elections.git

%cd Stance-Detection-Ghana-2020-Elections/

fatal: destination path 'Stance-Detection-Ghana-2020-Elections' already exists and is not an empty directory.
/content/Stance-Detection-Ghana-2020-Elections


In [2]:
!pip3 install config

In [3]:
!pip install vaderSentiment

In [4]:
import pandas as pd
import glob
import sys
sys.path.append('../../')
import config
import warnings
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report,f1_score

In [5]:
DATA_FILE_PATH = '/content/Stance-Detection-Ghana-2020-Elections/Dataset/'
CLEAN_DATA_FILE_NAME = '01EDA_All_Model_Tweets_Ghana_2020_Elections_L.csv'
TOKENIZED_DATA_FILE_NAME = '02TPP_All_Model_Tweets_Ghana_2020_Elections_L.csv'
pd.set_option('display.max_colwidth', None)
RANDOM_STATE = 42

# Load Data

In [6]:
df = pd.read_csv(DATA_FILE_PATH + TOKENIZED_DATA_FILE_NAME)

In [7]:
pd.set_option('display.max_colwidth', None)

In [8]:
df.head()

,ID,Tweet,Candidate,Stance,Preproc_Tweet,Tokens
0,2,"@NAkufoAddo H. E Nana please follow up on this FAKE NEWS, LIES and sue BBC for peddling false information. Hold alÃ¢â¬Â¦ https://t.co/SdwnMsxeUV",Nana,0.0,nana h e nana please follow up on this fake news lies and sue bbc for peddling false information hold al,"['nana', 'h', 'e', 'nana', 'pleas', 'follow', 'up', 'on', 'thi', 'fake', 'news', 'lie', 'and', 'sue', 'bbc', 'for', 'peddl', 'fals', 'inform', 'hold', 'al']"
1,6,"@ChipsJhay @TruthFreema @AOuattara_PRCI @NAkufoAddo No, you haven't. He's using his freedom of speech as it is. He's covered.",Nana,0.0,nana no you haven't he's using his freedom of speech as it is he's covered,"['nana', 'no', 'you', ""haven't"", ""he'"", 'use', 'hi', 'freedom', 'of', 'speech', 'as', 'it', 'is', ""he'"", 'cover']"
2,7,@IssahGerard @HEgyiri @NAkufoAddo Please who is this and how is he your in law??,Nana,0.0,nana please who is this and how is he your in law,"['nana', 'pleas', 'who', 'is', 'thi', 'and', 'how', 'is', 'he', 'your', 'in', 'law']"
3,17,@Kwame34580189 @bukolasaraki @NAkufoAddo Whatever.... Thought you had something there....,Nana,0.0,nana whatever thought you had something there,"['nana', 'whatev', 'thought', 'you', 'had', 'someth', 'there']"
4,18,@NAkufoAddo Remember that the good people of Gh have spoken therefore Soo be it,Nana,0.0,nana remember that the good people of gh have spoken therefore soo be it,"['nana', 'rememb', 'that', 'the', 'good', 'peopl', 'of', 'gh', 'have', 'spoken', 'therefor', 'soo', 'be', 'it']"


In [9]:
df = df[df.Stance != 0]

In [10]:
minimum_value = df['Stance'].value_counts().min()

In [11]:
def sample_minimum(df):
    return df.sample(minimum_value)

In [12]:
g = df.groupby('Stance')

In [13]:
new_df = g.apply(sample_minimum).reset_index(drop=True)
new_df.head()

,ID,Tweet,Candidate,Stance,Preproc_Tweet,Tokens
0,18363,@NAkufoAddo the mother serpent of corruption whoÃ¢â¬â¢s behaving as innnocent under anti- Corruption flower.\n\n~Martin A.Ã¢â¬Â¦ https://t.co/ZaTW75WkWc,Nana,-1.0,nana the mother serpent of corruption whos behaving as innnocent under anti- corruption flower\n\nmartin a,"['nana', 'the', 'mother', 'serpent', 'of', 'corrupt', 'who', 'behav', 'as', 'innnoc', 'under', 'anti', 'corrupt', 'flower', 'martin', 'a']"
1,80060,"@JDMahama Ghanaians don't want your ""free anything"" biaa cuz your so-called ""Free SHS"" is NOT up to Nana Addo's FreÃ¢â¬Â¦ https://t.co/TGhl3S8c84",John,-1.0,mahama ghanaians don't want your free anything biaa cuz your so-called free shs is not up to nana addo's fre,"['mahama', 'ghanaian', ""don't"", 'want', 'your', 'free', 'anyth', 'biaa', 'cuz', 'your', 'so-cal', 'free', 'sh', 'is', 'not', 'up', 'to', 'nana', ""addo'"", 'fre']"
2,12015,@TogbeDzatse @NAkufoAddo To still be number 1 in stealing Ghanaian hard earned money.,Nana,-1.0,nana to still be number in stealing ghanaian hard earned money,"['nana', 'to', 'still', 'be', 'number', 'in', 'steal', 'ghanaian', 'hard', 'earn', 'money']"
3,2035,@GHOneTV @NAkufoAddo @JDMahama Bias media you will continue to surfer under akuffo addo asotorwo,Nana,-1.0,nana mahama bias media you will continue to surfer under akuffo addo asotorwo,"['nana', 'mahama', 'bia', 'media', 'you', 'will', 'continu', 'to', 'surfer', 'under', 'akuffo', 'addo', 'asotorwo']"
4,73323,"@Renowned_beast @Supremo06903924 @JDMahama Wait, in 2012 did u see npp behaving like this with useless press conferÃ¢â¬Â¦ https://t.co/AXuGHEszS0",John,-1.0,mahama wait in did u see npp behaving like this with useless press confer,"['mahama', 'wait', 'in', 'did', 'u', 'see', 'npp', 'behav', 'like', 'thi', 'with', 'useless', 'press', 'confer']"


In [14]:
new_df['Stance'].value_counts()

-1.0    1030
 1.0    1030
Name: Stance, dtype: int64

### For VADERSentiment

In [15]:
analyser = SentimentIntensityAnalyzer()


In [16]:
def vader_sentiment(text):
    score = analyser.polarity_scores(text)
    if score['compound'] > 0:
        result = 1
    else:
        result = -1
    return result

### For TextBlob

In [17]:
def text_blob(text):
    score = TextBlob(text).sentiment.polarity
    if score > 0:
        result = 1
    else:
        result = -1
    return result

### Analysis and Evaluation


In [18]:
new_df['vader_sent'] = new_df['Tokens'].apply(vader_sentiment)
new_df['textblob_sent'] = new_df['Tokens'].apply(text_blob)


In [19]:
col = ['vader_sent','textblob_sent']
def pd_score(df,col,target):
    score = []
    for items in col:
        rate = accuracy_score(target, df[items])
        score.append(rate)
    scores = pd.Series(score, index=['VADER', 'TextBlob'])
    return scores
    

In [20]:
scores = pd_score(new_df,col,new_df['Stance'])

In [21]:
rate = accuracy_score(new_df['Stance'], new_df['vader_sent'])

In [22]:
rate

0.7694174757281553

In [23]:
rate = accuracy_score(new_df['Stance'], new_df['textblob_sent'])

In [24]:
rate

0.712135922330097

In [25]:
new_df.head()

,ID,Tweet,Candidate,Stance,Preproc_Tweet,Tokens,vader_sent,textblob_sent
0,18363,@NAkufoAddo the mother serpent of corruption whoÃ¢â¬â¢s behaving as innnocent under anti- Corruption flower.\n\n~Martin A.Ã¢â¬Â¦ https://t.co/ZaTW75WkWc,Nana,-1.0,nana the mother serpent of corruption whos behaving as innnocent under anti- corruption flower\n\nmartin a,"['nana', 'the', 'mother', 'serpent', 'of', 'corrupt', 'who', 'behav', 'as', 'innnoc', 'under', 'anti', 'corrupt', 'flower', 'martin', 'a']",-1,-1
1,80060,"@JDMahama Ghanaians don't want your ""free anything"" biaa cuz your so-called ""Free SHS"" is NOT up to Nana Addo's FreÃ¢â¬Â¦ https://t.co/TGhl3S8c84",John,-1.0,mahama ghanaians don't want your free anything biaa cuz your so-called free shs is not up to nana addo's fre,"['mahama', 'ghanaian', ""don't"", 'want', 'your', 'free', 'anyth', 'biaa', 'cuz', 'your', 'so-cal', 'free', 'sh', 'is', 'not', 'up', 'to', 'nana', ""addo'"", 'fre']",1,1
2,12015,@TogbeDzatse @NAkufoAddo To still be number 1 in stealing Ghanaian hard earned money.,Nana,-1.0,nana to still be number in stealing ghanaian hard earned money,"['nana', 'to', 'still', 'be', 'number', 'in', 'steal', 'ghanaian', 'hard', 'earn', 'money']",-1,-1
3,2035,@GHOneTV @NAkufoAddo @JDMahama Bias media you will continue to surfer under akuffo addo asotorwo,Nana,-1.0,nana mahama bias media you will continue to surfer under akuffo addo asotorwo,"['nana', 'mahama', 'bia', 'media', 'you', 'will', 'continu', 'to', 'surfer', 'under', 'akuffo', 'addo', 'asotorwo']",-1,-1
4,73323,"@Renowned_beast @Supremo06903924 @JDMahama Wait, in 2012 did u see npp behaving like this with useless press conferÃ¢â¬Â¦ https://t.co/AXuGHEszS0",John,-1.0,mahama wait in did u see npp behaving like this with useless press confer,"['mahama', 'wait', 'in', 'did', 'u', 'see', 'npp', 'behav', 'like', 'thi', 'with', 'useless', 'press', 'confer']",-1,-1


In [26]:
confusion_matrix(new_df['Stance'], new_df['vader_sent'])

array([[804, 226],
       [249, 781]])

In [27]:
confusion_matrix(new_df['Stance'], new_df['textblob_sent'])

array([[756, 274],
       [319, 711]])

In [28]:
print(classification_report(new_df['Stance'], new_df['vader_sent'], digits=4))

              precision    recall  f1-score   support

        -1.0     0.7635    0.7806    0.7720      1030
         1.0     0.7756    0.7583    0.7668      1030

    accuracy                         0.7694      2060
   macro avg     0.7696    0.7694    0.7694      2060
weighted avg     0.7696    0.7694    0.7694      2060



In [29]:
print(classification_report(new_df['Stance'], new_df['textblob_sent'], digits=4))

              precision    recall  f1-score   support

        -1.0     0.7033    0.7340    0.7183      1030
         1.0     0.7218    0.6903    0.7057      1030

    accuracy                         0.7121      2060
   macro avg     0.7125    0.7121    0.7120      2060
weighted avg     0.7125    0.7121    0.7120      2060

